---
## BU BÖLÜMDE "byte akışı" nın içerisinde HTTP başlıkları ve gövdeleri yerleştireceğiz.
---

# 🧩 Aşama C — TCP Echo’dan HTTP’ye: İlk “HTTP/1.1 200 OK” yanıtımız.

### 💡 Amaç:

* Tarayıcıdan gelen bir HTTP isteğini (ör. GET /) almak
> ve düzgün formatta bir HTTP yanıtı (ör. 200 OK) döndürmek.

Yani:
```bash
İstek  →  GET / HTTP/1.1
Yanıt  →  HTTP/1.1 200 OK
          Content-Type: text/html
          
          <h1>Merhaba Dünya</h1>


### 🧠 Mantık Adımları

* Tarayıcı bir TCP bağlantısı kurar ve HTTP isteğini düz metin olarak yollar.
Örneğin:
```bash
GET / HTTP/1.1
Host: 127.0.0.1:8080
User-Agent: Chrome
Accept: text/html


* Biz recv() ile bu metni okuruz.

* Bu metnin içinden “istek satırını” (örn. GET / HTTP/1.1) yakalarız.

* send() ile tarayıcıya şu yapıda bir yanıt göndeririz:
```bash
HTTP/1.1 200 OK
Content-Type: text/html
Content-Length: <gövde uzunluğu>

<body>...</body>
```
* (boş satır gövdenin başladığını belirtir!)


## ⚙️ Kod — Minimal HTTP Sunucu

In [ ]:
import socket

HOST = "127.0.0.1"
PORT = 8080
BUF = 4096

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"🌐 Sunucu çalışıyor: http://{HOST}:{PORT}")

    while True:
        client_socket, client_addr = srv.accept()
        print(f"🔌 Bağlantı alındı: {client_addr}")

        # 1️⃣ HTTP isteğini oku
        request = client_socket.recv(BUF).decode("utf-8")
        print("📩 Gelen İstek:\n", request)

        # 2️⃣ Basit HTML gövde oluştur
        body = "<h1>Merhaba Dünya! 🌍</h1>"
        response = f"""HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: {len(body)}

{body}"""

        # 3️⃣ Yanıtı gönder
        client_socket.sendall(response.encode("utf-8"))

        # 4️⃣ Bağlantıyı kapat
        client_socket.close()


### 🧩 Test Etme

* Kodu çalıştır:

>python http_server.py


* Tarayıcıya şunu yaz:

> http://127.0.0.1:8080


* Ekranda şu yazıyı görmelisin:

> Merhaba Dünya! 🌍

**Terminalde ise tarayıcıdan gelen ham HTTP isteği görünür:**

```bash
📩 Gelen İstek:
GET / HTTP/1.1
Host: 127.0.0.1:8080
User-Agent: Mozilla/5.0 ...
Accept: text/html


## 🔍 Ne Yaptık?

* Aslında şu anda tamamen kendi API’nı yazdık:

* Tarayıcıdan gelen istekleri (request) alıyoruz,

İçeriği çözüyor ve

Formatlı bir HTTP yanıtı oluşturuyoruz.

* FastAPI’nin veya Flask’ın temeli tam olarak bu.
* Sen şu anda onların “çekirdeğini” çıplak hâliyle kodladık.

---------

# 🧩 Aşama D — Mini Router Sistemi (Kendi API’miz şekilleniyor)

### 💡 Amaç:

Tarayıcıdan veya bir HTTP istemcisinden gelen URL yoluna göre farklı yanıtlar döndürmek.

| İstek    | Yanıt              |
| -------- | ------------------ |
| `/`      | “Ana sayfa” HTML   |
| `/hello` | “Merhaba!” mesajı  |
| `/data`  | JSON veri döndürme |


### 🧠 Mantık

* recv() ile gelen HTTP isteğini okuruz.

* İlk satırı (GET /hello HTTP/1.1) parçalayarak yolu (path) çıkarırız.

* if veya dict tabanlı bir router sistemi kurarız.

* İlgili içeriği döndürürken Content-Type’a dikkat ederiz:

>text/html → sayfa

>application/json → API cevabı

## ⚙️ Kod — Mini Routerlı HTTP Sunucu

In [ ]:
import socket
import json

HOST = "127.0.0.1"
PORT = 8080
BUF = 4096

def handle_request(path):
    """İstek yoluna göre yanıt döndürür"""
    if path == "/":
        body = "<h1>Ana Sayfa 🏠</h1><p>Hoş geldin, kendi API’m çalışıyor!</p>"
        content_type = "text/html"
    elif path == "/hello":
        body = "<h1>Merhaba! 👋</h1><p>Bu, kendi router’ımızdan dönen bir yanıt.</p>"
        content_type = "text/html"
    elif path == "/data":
        body = json.dumps({"status": "ok", "message": "API’den selamlar", "version": 1.0}, ensure_ascii=False)
        content_type = "application/json"
    else:
        body = "<h1>404 Not Found 🚫</h1><p>Bu sayfa bulunamadı.</p>"
        content_type = "text/html"

    return body, content_type


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as srv:
    srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    srv.bind((HOST, PORT))
    srv.listen(1)
    print(f"🚀 Sunucu dinlemede: http://{HOST}:{PORT}")

    while True:
        client_socket, client_addr = srv.accept()
        request = client_socket.recv(BUF).decode("utf-8")
        if not request:
            client_socket.close()
            continue

        # 1️⃣ İlk satırı al (ör: "GET /hello HTTP/1.1")
        request_line = request.splitlines()[0]
        print(f"📩 İstek satırı: {request_line}")

        try:
            method, path, version = request_line.split()
        except ValueError:
            client_socket.close()
            continue

        # 2️⃣ İçeriği oluştur
        body, content_type = handle_request(path)

        # 3️⃣ HTTP yanıtını hazırla
        response = f"""HTTP/1.1 200 OK
Content-Type: {content_type}; charset=utf-8
Content-Length: {len(body)}

{body}"""

        # 4️⃣ Gönder ve kapat
        client_socket.sendall(response.encode("utf-8"))
        client_socket.close()


🚀 Sunucu dinlemede: http://127.0.0.1:8080
📩 İstek satırı: GET / HTTP/1.1
📩 İstek satırı: GET /favicon.ico HTTP/1.1
📩 İstek satırı: GET /data HTTP/1.1
📩 İstek satırı: GET /favicon.ico HTTP/1.1


## 🔬 Ne Oldu?

Artık:

* Tarayıcı isteğini okuyoruz (GET /data HTTP/1.1)

* Yolu çıkarıyoruz (/data)

- İlgili fonksiyonu çalıştırıyoruz (handle_request)

* İçeriği formatlı şekilde döndürüyoruz

* Bu yapı, Flask’in @app.route() mekanizmasının ilkel hâlidir.

Yani FastAPI’nin içinde olan “routing sistemi”ni biz kendi elimizlw inşa ettik.

---
# 🧩 Kendi API’mız — Geliştirme Süreci  
📂 **Kendi API'mız\- API Geliştirne Adımları -`**



## ⚙️ Yaptıklarımız

- **Socket Temelleri**  
  TCP bağlantısının nasıl kurulduğunu öğrendik.  
  `socket()`, `bind()`, `listen()`, `accept()`, `recv()`, `send()` işlemlerini uyguladık.  
  İstemci–sunucu iletişimini kurduk.

- **Echo Server (Temel Refleks)**  
  Sunucuya gelen veriyi aynen geri gönderen bir TCP servis yazdık.  
  Veri kanalının (byte akışının) nasıl çalıştığını gözlemledik.

- **HTTP Katmanı (Mini Web Sunucu)**  
  Gelen ham TCP isteğini okuyup “HTTP/1.1 200 OK” cevabını formatladık.  
  Tarayıcıyla doğrudan konuşabilen bir mini web sunucu oluşturduk.

- **Mini Router Sistemi (Kendi Framework Çekirdeği)**  
  `/`, `/hello`, `/data` gibi URL yollarına göre farklı içerikler döndüren sistem kurduk.  
  HTML ve JSON içerik tiplerini yönettik.  
  Aslında sıfırdan bir “mini FastAPI” çekirdeği oluşturduk.



## 🚀 Bundan Sonra Ne Yapacağız

Artık **socket** aşaması tamam.  
Bundan sonraki adımlarda kendi API’mizi geliştirmeye devam edeceğiz.

### 📍 Devam Edilecek Adres:


>###  C:\Users\hdgn5\OneDrive\Masaüstü\Kendi API'mız- API Geliştirne Adımları -